### Project 6 - Exceptions

Suppose we have a Widget online sales application and we are writing the backend for it. We want a base `WidgetException` class that we will use as the base class for all our custom exceptions we raise from our Widget application.

Furthermore we have determined that we will need the following categories of exceptions:

```
1. Supplier exceptions
    a. Not manufactured anymore
    b. Production delayed
    c. Shipping delayed
    
2. Checkout exceptions
    a. Inventory type exceptions
        - out of stock
    b. Pricing exceptions
        - invalid coupon code
        - cannot stack coupons
```

Write an exception class hierarchy to capture this. In addition, we would like to implement the following functionality:
* implement separate internal error message and user error message
* implement an http status code associated to each exception type (keep it simple, use a 500 (server error) error for everything except invalid coupon code, and cannot stack coupons, these can be 400 (bad request)
* implement a logging function that can be called to log the exception details, time it occurred, etc.
* implement a function that can be called to produce a json string containing the exception details you want to display to your user (include http status code (e.g. 400), the user error message, etc)

##### Bonus

Log the traceback too - you'll have to do a bit of research for that! 

I'm going to use the `TracebackException` class in the `traceback` module:

https://docs.python.org/3/library/traceback.html#tracebackexception-objects

In particular, look at the class method `from_exception` (and remember that inside your exception class, the exception will be `self`!) and the `format` instance method. That method returns a generator, so you'll need to `list` it to print out everything in that traceback.

Good luck!

In [1]:
from http import HTTPStatus
import json, datetime, traceback

In [2]:
class WidgetException(Exception):
    internal_message = 'Widget Exception'
    user_messsage = 'Internal error.'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
    def __init__(self, *args, user_message=None):
        if args:
            self.internal_message = args[0]
            super().__init__(*args)
        else:
            super().__init__(self.internal_message)
        self.user_message = user_message if user_message is not None else self.internal_message
    
    def log_exception(self):
        exception = {'type': type(self).__name__,
                     'message': self.internal_message,
                     'args': self.args[1:],
                     'traceback': list(self.traceback)}
        print(f'Exception: {datetime.datetime.utcnow().isoformat()}: {exception}')
    
    def to_json(self):
        return json.dumps({'code': self.http_status.value, 
                           'message': f'{self.http_status.phrase}: ' + self.user_message, 
                           'category': type(self).__name__,
                           'time_utc': datetime.datetime.utcnow().isoformat()})
    
    @property
    def traceback(self):
        return traceback.TracebackException.from_exception(self).format()

In [3]:
class SupplierException(WidgetException):
    internal_message = 'Supplier Exception'
    
class NotManufacturedException(SupplierException):
    internal_message = 'Not Manufactured Exception'

class ProductionDelayedException(SupplierException):
    internal_message = 'Production Delayed Exception'

class ShippingDelayedException(SupplierException):
    internal_message = 'Shipping Delayed Exception'

class CheckoutException(WidgetException):
    internal_message = 'Checkout Exception'

class InventoryException(CheckoutException):
    internal_message = 'Inventory Exception'

class OutOfStockException(InventoryException):
    internal_message = 'Out of Stock Exception'

class PricingException(CheckoutException):
    internal_message = 'Pricing Exception'
    
class InvalidCouponCodeException(PricingException):
    internal_message = 'Invalid Coupon Code Exception'
    http_status = HTTPStatus.BAD_REQUEST
    
class CannotStackCouponException(PricingException):
    internal_message = 'cannot stack coupons'
    http_status = HTTPStatus.BAD_REQUEST

In [4]:
ex1 = WidgetException('some custom message', 10, 100)
ex2 = WidgetException(user_message='A custom user message.')
ex1.log_exception()

Exception: 2023-05-21T07:32:04.574989: {'type': 'WidgetException', 'message': 'some custom message', 'args': (10, 100), 'traceback': ["WidgetException: ('some custom message', 10, 100)\n"]}


In [5]:
ex2.log_exception()

Exception: 2023-05-21T07:32:04.629808: {'type': 'WidgetException', 'message': 'Widget Exception', 'args': (), 'traceback': ['WidgetException: Widget Exception\n']}


In [6]:
try:
    raise CannotStackCouponException()
except WidgetException as ex:
    ex.log_exception()
    raise

Exception: 2023-05-21T07:32:04.672561: {'type': 'CannotStackCouponException', 'message': 'cannot stack coupons', 'args': (), 'traceback': ['Traceback (most recent call last):\n', '  File "<ipython-input-6-190053311f97>", line 2, in <module>\n    raise CannotStackCouponException()\n', 'CannotStackCouponException: cannot stack coupons\n']}


CannotStackCouponException: cannot stack coupons

In [7]:
e = WidgetException('same custom message for log and user')
e.log_exception()

Exception: 2023-05-21T07:32:16.167374: {'type': 'WidgetException', 'message': 'same custom message for log and user', 'args': (), 'traceback': ['WidgetException: same custom message for log and user\n']}


In [8]:
json.loads(e.to_json())

{'code': 500,
 'message': 'Internal Server Error: same custom message for log and user',
 'category': 'WidgetException',
 'time_utc': '2023-05-21T07:32:16.174684'}

In [9]:
e = WidgetException('custom internal message', user_message='custom user message')
e.log_exception()

Exception: 2023-05-21T07:32:16.230101: {'type': 'WidgetException', 'message': 'custom internal message', 'args': (), 'traceback': ['WidgetException: custom internal message\n']}


In [10]:
e.to_json()

'{"code": 500, "message": "Internal Server Error: custom user message", "category": "WidgetException", "time_utc": "2023-05-21T07:32:16.242028"}'

In [11]:
try:
    raise WidgetException('custom error message')
except WidgetException as ex:
    print(repr(ex.__traceback__))

In [12]:
try:
    raise ValueError
except ValueError:
    try:
        raise WidgetException('custom error message')
    except WidgetException as ex:
        print(list(traceback.TracebackException.from_exception(ex).format()))

['Traceback (most recent call last):\n', '  File "<ipython-input-12-2a9225338511>", line 2, in <module>\n    raise ValueError\n', 'ValueError\n', '\nDuring handling of the above exception, another exception occurred:\n\n', 'Traceback (most recent call last):\n', '  File "<ipython-input-12-2a9225338511>", line 5, in <module>\n    raise WidgetException(\'custom error message\')\n', 'WidgetException: custom error message\n']


In [13]:
try:
    raise WidgetException('custom internal message', user_message='custom user message')
except WidgetException as ex:
    ex.log_exception()
    print('------------')
    print(ex.to_json())

Exception: 2023-05-21T07:32:16.346073: {'type': 'WidgetException', 'message': 'custom internal message', 'args': (), 'traceback': ['Traceback (most recent call last):\n', '  File "<ipython-input-13-5f5ce724ebb7>", line 2, in <module>\n    raise WidgetException(\'custom internal message\', user_message=\'custom user message\')\n', 'WidgetException: custom internal message\n']}
------------
{"code": 500, "message": "Internal Server Error: custom user message", "category": "WidgetException", "time_utc": "2023-05-21T07:32:16.346189"}


In [14]:
e = InvalidCouponCodeException('User tried to use an old coupon', user_message='Sorry. This coupon has expired.')
e.log_exception()

Exception: 2023-05-21T07:32:16.359730: {'type': 'InvalidCouponCodeException', 'message': 'User tried to use an old coupon', 'args': (), 'traceback': ['InvalidCouponCodeException: User tried to use an old coupon\n']}


In [15]:
e.to_json()

'{"code": 400, "message": "Bad Request: Sorry. This coupon has expired.", "category": "InvalidCouponCodeException", "time_utc": "2023-05-21T07:32:16.370365"}'

In [16]:
try:
    raise ValueError
except ValueError:
    try:
        raise InvalidCouponCodeException(
            'User tried to use an old coupon', user_message='Sorry. This coupon has expired.'
        )
    except InvalidCouponCodeException as ex:
        ex.log_exception()
        print('------------')
        print(ex.to_json())
        print('------------')
        print(''.join(ex.traceback))

Exception: 2023-05-21T07:32:16.422647: {'type': 'InvalidCouponCodeException', 'message': 'User tried to use an old coupon', 'args': (), 'traceback': ['Traceback (most recent call last):\n', '  File "<ipython-input-16-a8d1ab2e776f>", line 2, in <module>\n    raise ValueError\n', 'ValueError\n', '\nDuring handling of the above exception, another exception occurred:\n\n', 'Traceback (most recent call last):\n', '  File "<ipython-input-16-a8d1ab2e776f>", line 5, in <module>\n    raise InvalidCouponCodeException(\n', 'InvalidCouponCodeException: User tried to use an old coupon\n']}
------------
{"code": 400, "message": "Bad Request: Sorry. This coupon has expired.", "category": "InvalidCouponCodeException", "time_utc": "2023-05-21T07:32:16.423483"}
------------
Traceback (most recent call last):
  File "<ipython-input-16-a8d1ab2e776f>", line 2, in <module>
    raise ValueError
ValueError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):